TDS Project: Dataset Selection, EDA, and Basic Model Analysis


Mahmud Hassan 208247825
Khalil Hamody 212923775


In [ ]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Setting plot styles
sns.set_theme(style="whitegrid")
%matplotlib inline

In [ ]:
# Loading the dataset
data = pd.read_csv('data/data.csv')

# Displaying the first few rows
data.head()

Check Dataset Information
Understand the structure, types of data, and missing values.

In [ ]:
# Display basic information about the dataset
print(data.info())

# Summary statistics of numerical columns
print(data.describe())

# Check for missing values
print(data.isnull().sum())
